In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import joblib
from torch.nn.functional import softmax

# Pre-load assets once (outside the prediction function)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
le = joblib.load("model_assets/label_encoder.joblib")

# Load model with optimizations
loaded_model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(le.classes_)
).to(device)
loaded_model.load_state_dict(torch.load("model_assets/best_model.pt")["model_state_dict"])

# Apply optimizations
loaded_model.eval()
torch.backends.cudnn.benchmark = True  # Enable cuDNN auto-tuner
if torch.cuda.is_available():
    loaded_model = torch.compile(loaded_model)  # PyTorch 2.0 compiler

def predict(text, model=loaded_model, tokenizer=tokenizer, label_encoder=le):
    """Returns prediction with confidence score"""
    with torch.no_grad(), torch.inference_mode():
        inputs = tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=128,
            return_tensors="pt"
        ).to(device)

        # Use fused kernels where available
        with torch.backends.cuda.sdp_kernel(enable_flash=True):
            outputs = model(**inputs)

        # Get probabilities and prediction
        probs = softmax(outputs.logits, dim=1)
        confidence, pred_idx = torch.max(probs, dim=1)

        return {
            'prediction': label_encoder.inverse_transform(pred_idx.cpu().numpy())[0],
            'confidence': round(confidence.item(), 4)  # Rounded to 4 decimal places
        }

def predict_batch(texts, batch_size=32):
    """Batch prediction with confidence scores"""
    encodings = tokenizer(
        texts,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors="pt",
        add_special_tokens=True
    )

    dataset = torch.utils.data.TensorDataset(
        encodings['input_ids'],
        encodings['attention_mask']
    )

    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

    results = []
    with torch.no_grad(), torch.inference_mode():
        for batch in loader:
            inputs = {k: v.to(device) for k, v in zip(['input_ids', 'attention_mask'], batch)}
            with torch.backends.cuda.sdp_kernel(enable_flash=True):
                outputs = loaded_model(**inputs)

            # Process batch with confidence
            probs = softmax(outputs.logits, dim=1)
            confidences, pred_indices = torch.max(probs, dim=1)
            predictions = label_encoder.inverse_transform(pred_indices.cpu().numpy())

            results.extend([
                {
                    'prediction': pred,
                    'confidence': round(conf.item(), 4)
                }
                for pred, conf in zip(predictions, confidences)
            ])

    return results

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
result = predict("i have issue in my fammily")
print(f"Prediction: {result['prediction']} (Confidence: {result['confidence']:.2%})")


/usr/lib/python3.11/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)


Prediction: Student Affairs (Confidence: 24.73%)


In [ ]:
result = predict("I was really excited to depart on a weekend trip with my friends, but the transportation ended up being a nightmare. Our bus infract down on the side of the road, and we complete up being stranded for hours.")
print(f"Prediction: {result['prediction']} (Confidence: {result['confidence']:.2%})")


/usr/lib/python3.11/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)


Prediction: Activities and Travelling (Confidence: 72.42%)


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
import zipfile
import os

# Path to the ZIP file in your Drive
zip_path = '/content/drive/MyDrive/Fast_genre.zip'

# Destination folder
output_dir = '/content/model_assets'
os.makedirs(output_dir, exist_ok=True)


In [ ]:
from tqdm import tqdm

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()

    for file in tqdm(file_list, desc="Extracting files"):
        zip_ref.extract(member=file, path=output_dir)

Extracting files: 100%|██████████| 6/6 [00:20<00:00,  3.49s/it]


run that the upcoming only for my team to load data

---



In [ ]:
import gdown
import zipfile
import os
from tqdm import tqdm

# Step 1: Download the ZIP file from Google Drive (shared link)
file_id = '1juliMp2yJwMe08Yh0s2uYpu_3U4miUAR'  # Extracted from shared URL
zip_path = 'downloaded_file.zip'

gdown.download(f'https://drive.google.com/uc?id={file_id}', zip_path, quiet=False)

# Step 2: Create the output directory
output_dir = 'model_assets'
os.makedirs(output_dir, exist_ok=True)

# Step 3: Unzip the file with progress
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()
    for file in tqdm(file_list, desc="Extracting files"):
        zip_ref.extract(member=file, path=output_dir)

print(f"\n✅ Done! Files extracted to '{output_dir}'")


Downloading...
From (original): https://drive.google.com/uc?id=1juliMp2yJwMe08Yh0s2uYpu_3U4miUAR
From (redirected): https://drive.google.com/uc?id=1juliMp2yJwMe08Yh0s2uYpu_3U4miUAR&confirm=t&uuid=d69c97d2-5f70-4b7f-a007-1bb85bccbd65
To: /content/downloaded_file.zip
100%|██████████| 1.05G/1.05G [00:14<00:00, 72.1MB/s]
Extracting files: 100%|██████████| 6/6 [00:21<00:00,  3.55s/it]


✅ Done! Files extracted to 'model_assets'


In [ ]:
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()

    for file in tqdm(file_list, desc="Extracting files"):
        zip_ref.extract(member=file, path=output_dir)

Extracting files: 100%|██████████| 6/6 [00:20<00:00,  3.48s/it]
